<a href="https://colab.research.google.com/github/ioloi1128/ML100Days/blob/main/CDA_week2_ex1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

install

In [ ]:
!pip install numpy

In [ ]:
!pip install matplotlib

In [ ]:
!pip install pandas

In [ ]:
!pip install zarr

In [ ]:
!pip install fsspec

In [ ]:
!pip install gcsfs

In [ ]:
!pip install xarray[complete]

define

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import zarr
import fsspec

In [ ]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df.head()

In [ ]:
df_ta = df.query("activity_id=='CMIP' & table_id == 'Amon' & variable_id == 'tas' & experiment_id == 'historical'")
df_ta

In [ ]:
df_ta_ncar = df_ta.query('institution_id == "NCAR"')
df_ta_ncar

In [ ]:
# get the path to a specific zarr store (the first one from the dataframe above)
zstore = df_ta_ncar.zstore.values[-1]
print(zstore)

In [15]:
# create a mutable-mapping-style interface to the store
mapper = fsspec.get_mapper(zstore)

In [ ]:
# open it using xarray and zarr
ds = xr.open_zarr(mapper, consolidated=True)
ds

save data

In [17]:
ds.to_netcdf(path="saved_NCAR.ts.nc")

In [ ]:
from google.colab import files

with open('saved_NCAR.ts.nc', 'w') as f:
  f.write('some content')

files.download('saved_NCAR.ts.nc')

plot and analysis the data

In [ ]:
ds.tas.sel(time='1960-07').squeeze().plot()

In [ ]:
df_area = df.query("variable_id == 'areacella' & source_id == 'CESM2'")
ds_area = xr.open_zarr(fsspec.get_mapper(df_area.zstore.values[0]), consolidated=True)
ds_area

In [ ]:
total_area = ds_area.areacella.sum(dim=['lon', 'lat'])
ta_timeseries = (ds.tas * ds_area.areacella).sum(dim=['lon', 'lat']) / total_area
ta_timeseries

In [ ]:
%time ta_timeseries.load()

In [ ]:
ta_timeseries.plot(label='monthly')
ta_timeseries.rolling(time=12).mean().plot(label='12 month rolling mean')
plt.legend()
plt.title('Global Mean Surface Air Temperature')